In [15]:
import pandas as pd
import numpy as np
import h5py
from astropy.io import fits
from astropy.table import Table
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [2]:
with h5py.File('../data/JDrgb_14m_hasrv.hdf5', 'r') as f:
    columns_group = f['table/columns']
    col_names = list(columns_group.keys())
    data_dict = {}

    for col in tqdm(col_names, desc="Reading HDF5 columns"):
        dataset = columns_group[col]
        subkey = list(dataset.keys())[0]
        data = dataset[subkey][:]
        if data.dtype.kind == 'S':
            data = data.astype(str)
        data_dict[col] = data

df_vis = pd.DataFrame(data_dict)
print(f"Loaded df_vis with shape {df_vis.shape}")

Reading HDF5 columns: 100%|██████████| 30/30 [00:00<00:00, 57.80it/s]


Loaded df_vis with shape (14485519, 30)


In [3]:
print("Loading FITS...")
with fits.open('../data/table_2_catwise.fits.gz', memmap=True) as hdul:
    data = hdul[1].data
    df_RGB = pd.DataFrame({
        col.name: data[col.name].byteswap().newbyteorder() if data[col.name].dtype.byteorder == '>' else data[col.name]
        for col in hdul[1].columns
    })

df_rgb_subset = df_RGB[['source_id', 'catwise_w1', 'catwise_w2', 'mh_xgboost', 'teff_xgboost', 'logg_xgboost']]


Loading FITS...


In [5]:
print("Merging dataframes...")
df_vis['source_id'] = df_vis['source_id'].astype(str)
df_rgb_subset = df_RGB[['source_id', 'catwise_w1', 'catwise_w2', 'mh_xgboost', 'teff_xgboost', 'logg_xgboost']].copy()
df_rgb_subset['source_id'] = df_rgb_subset['source_id'].astype(str)
df_merged = pd.merge(df_vis, df_rgb_subset, on='source_id', how='left')
print(f"Merged shape: {df_merged.shape}")


Merging dataframes...
Merged shape: (14485519, 35)


In [6]:
# Count how many rows have any missing values in the columns added from df_rgb_subset
num_missing = df_merged[['catwise_w1', 'catwise_w2', 'mh_xgboost', 'teff_xgboost', 'logg_xgboost']].isna().any(axis=1).sum()

print(f"[✓] Number of rows in df_vis without a match from df_rgb_subset: {num_missing}")


[✓] Number of rows in df_vis without a match from df_rgb_subset: 2507010


In [7]:
# Drop rows with missing values in any of the merged columns
df_merged = df_merged.dropna(subset=['catwise_w1', 'catwise_w2', 'mh_xgboost', 'teff_xgboost', 'logg_xgboost'])

print(f"[✓] Shape after dropping unmatched rows: {df_merged.shape}")


[✓] Shape after dropping unmatched rows: (11978509, 35)


In [8]:
print("Applying quality cuts...")

df_merged['parallax_over_error'] = df_merged['parallax'] / df_merged['parallax_error']

df_merged['MW1'] = df_merged['catwise_w1'] + 5 * np.log10(df_merged['parallax'] / 100)

df_merged['G'] = df_merged['phot_g_mean_mag']

df_merged['GBP'] = df_merged['phot_bp_mean_mag']

df_merged['W1'] = df_merged['catwise_w1']
df_merged['W2'] = df_merged['catwise_w2']

cut = (
    (df_merged['phot_g_mean_mag'] < 16) &
    (df_merged['parallax_over_error'] > 5) &
    (df_merged['teff_xgboost'] <= 5200) &
    (df_merged['logg_xgboost'] < 3.5) &
    (df_merged['MW1'] > (-0.3 - 0.006 * (5500 - df_merged['teff_xgboost']))) &
    (df_merged['MW1'] > (-0.01 * (5300 - df_merged['teff_xgboost']))) &
    ((df_merged['G'] - df_merged['W2']) < (0.2 + 0.77 * (df_merged['GBP'] - df_merged['W1'])))
)

df_cleaned = df_merged[cut].copy()
print(f"After cuts: {df_cleaned.shape}")

Applying quality cuts...
After cuts: (10456910, 41)


In [9]:
dwarf_galaxies = Table.read("../local_volume_database/data/dwarf_mw.csv")
globular_clusters = Table.read("../local_volume_database/data/gc_harris.csv")

dwarf_coords = SkyCoord(ra=dwarf_galaxies['ra'] * u.deg, dec=dwarf_galaxies['dec'] * u.deg)
gc_coords = SkyCoord(ra=globular_clusters['ra'] * u.deg, dec=globular_clusters['dec'] * u.deg)
known_coords = SkyCoord(
    ra=np.concatenate([dwarf_coords.ra.deg, gc_coords.ra.deg]) * u.deg,
    dec=np.concatenate([dwarf_coords.dec.deg, gc_coords.dec.deg]) * u.deg
)

print("Computing nearest matches...")
df_cleaned['ra'] = pd.to_numeric(df_cleaned['ra'], errors='coerce')
df_cleaned['dec'] = pd.to_numeric(df_cleaned['dec'], errors='coerce')
df_cleaned = df_cleaned.dropna(subset=['ra', 'dec'])

sample_coords = SkyCoord(ra=df_cleaned['ra'].values * u.deg, dec=df_cleaned['dec'].values * u.deg)

# Use efficient nearest neighbor match instead of all-pair separation
idx, sep2d, _ = sample_coords.match_to_catalog_sky(known_coords)

# Filter stars >1 degree away
df_final = df_cleaned[sep2d.deg > 1].copy()
print(f"After sky filtering: {df_final.shape}")

Computing nearest matches...
After sky filtering: (10060704, 41)


In [10]:
# saving the final DataFrame to a FITS file

from astropy.table import Table

# Convert source_id to string to prevent issues when saving to FITS
df_final['source_id'] = df_final['source_id'].astype(str)

# Convert to Astropy Table
table = Table.from_pandas(df_final)

# Save to FITS
output_fits_filename = '../data/vis_cleaned.fits'
table.write(output_fits_filename, format='fits', overwrite=True)

print(f"[✓] FITS file saved as '{output_fits_filename}'")


[✓] FITS file saved as '../data/vis_cleaned.fits'


In [13]:
pd.set_option('display.max_columns', None)
df_final.shape

(10060704, 41)

In [3]:
#loading

from astropy.table import Table

# Load the FITS file as an Astropy Table
table = Table.read("../data/vis_cleaned.fits", format='fits')

# Convert to a pandas DataFrame
df_vis = table.to_pandas()
df_vis['source_id'] = df_vis['source_id'].str.decode('utf-8').astype(np.int64)


In [4]:
df_vis.head()

,aom_xp,b,bp_rp,dec,e_aom_xp,e_logg_xp,e_moh_xp,e_teff_xp,fake_MG,l,logg_xp,moh_xp,parallax,parallax_error,phot_bp_mean_mag,phot_bp_rp_excess_factor,phot_g_mean_mag,phot_rp_mean_mag,pmdec,pmdec_error,pmra,pmra_error,ra,radial_velocity,radial_velocity_error,ruwe,snr_bp,snr_rp,source_id,teff_xp,catwise_w1,catwise_w2,mh_xgboost,teff_xgboost,logg_xgboost,parallax_over_error,MW1,G,GBP,W1,W2
0,0.0189,-48.572035,1.154534,0.335043,0.0222,0.1309,0.0717,53.5303,158.232347,176.739184,2.8993,-0.1789,1.407660,0.018947,10.750277,1.230957,10.253987,9.595743,-1.412098,0.016528,-0.711280,0.017718,45.136038,-0.738894,0.316921,1.036041,339.585517,704.910156,15741055975040,5096.609863,8.152,8.198,-0.144,5065.8,2.993,74.293738,-1.105511,10.253987,10.750277,8.152,8.198
1,0.1512,-48.171322,1.409290,0.736093,0.0502,0.1197,0.0759,44.4781,63.236201,176.483565,1.7891,-0.4944,0.500272,0.020692,11.150994,1.254608,10.508797,9.741704,1.594356,0.022822,3.309832,0.022959,45.305053,41.607450,1.323430,1.349013,254.753264,849.925883,66627828480768,4480.451172,7.891,7.964,-0.401,4499.0,1.916,24.176686,-3.612968,10.508797,11.150994,7.891,7.964
2,0.0036,-48.607026,1.189063,0.561503,0.0080,0.0742,0.0333,30.6331,293.932164,176.209301,3.0343,0.0696,2.177522,0.016049,11.169669,1.230755,10.651423,9.980606,-12.678339,0.013950,-4.869755,0.015797,44.866246,-32.461674,0.205614,1.018742,330.678438,796.410471,82467667849472,4890.509277,8.496,8.558,0.114,4938.3,3.203,135.681722,0.185813,10.651423,11.169669,8.496,8.558
3,0.2962,-48.727781,1.131072,0.689953,0.0328,0.1724,0.0808,59.5872,356.979240,175.755174,2.9970,-0.5701,0.485402,0.024241,14.816144,1.228919,14.332705,13.685072,-1.599436,0.021891,3.539184,0.027864,44.569524,91.295900,4.830661,1.176748,197.334453,241.162594,101193725229056,4889.291016,12.220,12.275,-0.478,4980.2,3.319,20.024124,0.650509,14.332705,14.816144,12.220,12.275
4,0.0726,-48.328584,1.328486,0.955080,0.0245,0.0762,0.0429,27.8866,143.555092,175.789759,2.5074,-0.1392,0.820288,0.017479,11.810220,1.246487,11.215262,10.481733,-0.291235,0.015982,3.382907,0.017619,44.868872,1.959265,0.328298,1.087555,312.296155,668.643675,130399502833792,4682.432129,8.728,8.815,-0.159,4643.6,2.398,46.929778,-1.702169,11.215262,11.810220,8.728,8.815


In [ ]:
# Select only RA and Dec columns
target_df = df_vis[['ra', 'dec']]

# Save to text file with no header, space-separated
target_df.to_csv('../data/target_vis.txt', sep=' ', index=False, header=False)

print("target_vis.txt file created successfully!")


target_vis.txt file created successfully!


## correct up to this point

- something is wrong with the way i am queruing - consider a smaller arcminute - go to arcseconds perhaps

In [ ]:
from astropy.io import fits

fits_file = '../data/vis_dis.fit'

with fits.open(fits_file) as hdul:
    # Print summary of all HDUs
    hdul.info()

    # Example: inspect the first table HDU (usually HDU[1])
    if len(hdul) > 1:
        print("\nHDU[1] header:")
        print(repr(hdul[1].header))

        print("\nHDU[1] data preview:")
        print(hdul[1].data[:5])  # Show first 5 rows


i dont know why the data i need isnt in this

In [14]:
df_vis_dis.head()

NameError: name 'df_vis_dis' is not defined

In [8]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# Convert to SkyCoord
coords_vis = SkyCoord(ra=df_vis['ra'].values * u.deg, dec=df_vis['dec'].values * u.deg)
coords_dis = SkyCoord(ra=df_vis_dis['RA_ICRS'].values * u.deg, dec=df_vis_dis['DE_ICRS'].values * u.deg)

# Match to nearest
idx, d2d, _ = coords_vis.match_to_catalog_sky(coords_dis)

# Find matches within 1 arcsecond
matched = d2d < 1.0 * u.arcsec
print(f"[✓] Position matches within 1 arcsec: {matched.sum()} / {len(df_vis)}")


[✓] Position matches within 1 arcsec: 102458 / 10060704


In [5]:
from tqdm import tqdm

# Optional: enable tqdm for pandas operations
tqdm.pandas(desc="Checking duplicates")

# Check full row duplicates using tqdm
duplicate_flags = df_vis_dis.progress_apply(lambda row: tuple(row) in set(), axis=1)  # Dummy to trigger tqdm
full_duplicates = df_vis_dis[df_vis_dis.duplicated()]
print(f"\n[✓] Full duplicated rows: {len(full_duplicates)}")
if not full_duplicates.empty:
    print(full_duplicates.head())

# Check for duplicates based on 'source_id', if present
if 'source_id' in df_vis_dis.columns:
    id_duplicates = df_vis_dis[df_vis_dis.duplicated('source_id')]
    print(f"[✓] Duplicated source_id entries: {len(id_duplicates)}")
    if not id_duplicates.empty:
        print(id_duplicates.head())


Checking duplicates: 100%|██████████| 84757989/84757989 [02:34<00:00, 549657.57it/s]



[✓] Full duplicated rows: 62175627
              Source    RA_ICRS   DE_ICRS        rpgeo      b_rpgeo  \
142  275118425722752  45.811539  1.617525   715.566040   510.238129   
145  275702541277440  45.817596  1.631048  2994.836670  2554.120610   
149  275771260754688  45.806771  1.636314  1432.225590  1277.350590   
172  298345608489344  44.408610  0.985043   454.192596   445.504211   
173  298349903957504  44.407948  0.985147   440.120392   429.566193   

      b_rpgeo_lc  
142  1274.047850  
145  3549.351810  
149  1632.537960  
172   461.881256  
173   452.042603  


In [7]:
from tqdm import tqdm

# Remove full-row duplicates
print(f"[→] Initial shape: {df_vis_dis.shape}")
df_vis_dis = df_vis_dis.drop_duplicates().reset_index(drop=True)
print(f"[✓] Removed full duplicates. New shape: {df_vis_dis.shape}")

# Remove duplicates based on 'Source', keeping the first
if 'Source' in df_vis_dis.columns:
    print(f"[→] Checking and removing duplicates based on 'Source'...")
    tqdm.pandas(desc="Removing Source duplicates")
    df_vis_dis = df_vis_dis.drop_duplicates(subset='Source', keep='first').reset_index(drop=True)
    print(f"[✓] Removed 'Source' duplicates. Final shape: {df_vis_dis.shape}")


[→] Initial shape: (84757989, 6)
[✓] Removed full duplicates. New shape: (22582362, 6)
[→] Checking and removing duplicates based on 'Source'...
[✓] Removed 'Source' duplicates. Final shape: (22582362, 6)


In [8]:
df_vis_dis.head()

,Source,RA_ICRS,DE_ICRS,rpgeo,b_rpgeo,b_rpgeo_lc
0,14190572136064,45.139716,0.317631,2733.080080,2314.287840,3166.355710
1,14190572136192,45.138628,0.318988,369.159943,350.101532,389.457489
2,14190572136320,45.148487,0.319526,1245.477420,1024.909060,1395.592290
3,14263587225600,45.134758,0.321555,238.853394,237.404816,240.157104
4,15083925341824,45.114580,0.350138,321.531372,301.825623,345.233154


In [9]:
from astropy.table import Table

# Convert to Astropy Table
table = Table.from_pandas(df_vis_dis)

# Save to FITS file
output_path = "../data/vis_distances.fits"
table.write(output_path, format='fits', overwrite=True)

print(f"[✓] Saved cleaned table to: {output_path}")

[✓] Saved cleaned table to: ../data/vis_distances.fits


In [1]:
from astropy.table import Table
import pandas as pd
import numpy as np

# Load vis_cleaned.fits
vis_cleaned_path = "../data/vis_cleaned.fits"
df_vis = Table.read(vis_cleaned_path, format='fits').to_pandas()
df_vis['source_id'] = df_vis['source_id'].str.decode('utf-8').astype(np.int64)
print(f"[✓] Loaded vis_cleaned: {df_vis.shape}")

# Load vis_distances.fits
vis_distances_path = "../data/vis_distances.fits"
df_dist = Table.read(vis_distances_path, format='fits').to_pandas()
print(f"[✓] Loaded vis_distances: {df_dist.shape}")


[✓] Loaded vis_cleaned: (10060704, 41)
[✓] Loaded vis_distances: (22582362, 6)


In [17]:
df_vis.shape

(14485519, 30)

In [3]:
df_dist.head()

,Source,RA_ICRS,DE_ICRS,rpgeo,b_rpgeo,b_rpgeo_lc
0,14190572136064,45.139716,0.317631,2733.080080,2314.287840,3166.355710
1,14190572136192,45.138628,0.318988,369.159943,350.101532,389.457489
2,14190572136320,45.148487,0.319526,1245.477420,1024.909060,1395.592290
3,14263587225600,45.134758,0.321555,238.853394,237.404816,240.157104
4,15083925341824,45.114580,0.350138,321.531372,301.825623,345.233154


In [4]:
# Rename 'Source' to 'source_id' in df_dist
df_dist = df_dist.rename(columns={"Source": "source_id"})

# Ensure source_id is numeric
df_vis['source_id'] = df_vis['source_id'].astype(np.int64)
df_dist['source_id'] = df_dist['source_id'].astype(np.int64)

# Merge on 'source_id'
df_merged = pd.merge(df_vis, df_dist, on='source_id', how='inner')
print(f"[✓] Merged shape (before deduplication): {df_merged.shape}")

# Remove duplicate rows if any
df_merged = df_merged.drop_duplicates()
print(f"[✓] Final shape after removing duplicates: {df_merged.shape}")


[✓] Merged shape (before deduplication): (102458, 46)
[✓] Final shape after removing duplicates: (102458, 46)


In [6]:
from astropy.table import Table
import numpy as np

# Save to table
source_id_table = Table()
source_id_table['Source'] = df_vis['source_id'].astype(np.int64)